In [ ]:
# Experiment no.09 [pilot]

In [11]:
# Imports

# PyTorch family
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch import optim

import torchvision
from torchvision import datasets, transforms
from torchvision.models import resnet101

# SnnTorch family
import snntorch as snn
from snntorch import spikeplot as splt
from snntorch import spikegen

# Utilities
import matplotlib.pyplot as plt
import numpy as np
import itertools

import sys
sys.path.append('/home/hwkang/jupyter/root/utility/')
from synthesize import generate_one_noisy_image
from preprocess import get_single_subset_by_ratio

In [7]:
# Datasets
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.CIFAR10(root=path_dataset,
                                 train=True,
                                 transform=transform,
                                 download=False)
test_dataset = datasets.CIFAR10(root=path_dataset,
                                 train=False,
                                 transform=transform,
                                 download=False)

In [8]:
# 빠른 실행을 위해 데이터셋 크기 축소
train_dataset = get_single_subset_by_ratio(train_dataset, 0.1)
test_dataset = get_single_subset_by_ratio(test_dataset, 0.1)

In [12]:
# DataLoaders
train_loader = DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True)
test_loader = DataLoader(test_dataset,
                         batch_size=64,
                         shuffle=False)

In [ ]:
# Network

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [17]:
# ResNet for classification
model_resnet = resnet101(weights=None, num_classes=10)
model_resnet = model_resnet.to(device)

In [19]:
# Hyperparameters
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_resnet.parameters(), lr=0.001, momentum=0.9)
num_epochs = 10

In [21]:
# Training Loop
for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        
        outputs = model_resnet(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {running_loss / len(train_loader):.4f}")

Epoch [1/10], Loss: 2.3639
Epoch [2/10], Loss: 2.3158
Epoch [3/10], Loss: 2.1680
Epoch [4/10], Loss: 2.1054
Epoch [5/10], Loss: 2.0988
Epoch [6/10], Loss: 2.0578
Epoch [7/10], Loss: 1.9266
Epoch [8/10], Loss: 1.8456
Epoch [9/10], Loss: 1.7811
Epoch [10/10], Loss: 1.7289


In [22]:
# Evaluation
# 테스트 성능 평가
model_resnet.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model_resnet(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Accuracy on test set: {100 * correct / total:.2f}%")

Accuracy on test set: 33.50%


In [ ]:
# SNN-CNN for classification